In [39]:
from datetime import datetime
import pandas as pd
import numpy as np


In [40]:
trades_file = "..\\..\\data\\moex\\SBER_240206_240606.csv"

dtype={
    '<DATE>': 'string',
    '<TIME>': 'string',
    '<LAST>': 'float',
    '<VOL>': 'int64',
    '<ID>': 'int64',
    '<OPER>': 'string',
}

trades = pd.read_csv(trades_file, delimiter=";", decimal=".", dtype=dtype)
trades['time'] = pd.to_datetime(trades['<DATE>'] + ' ' + trades['<TIME>'], format='%y%m%d %H%M%S')
trades.drop(['<DATE>', '<TIME>'], axis=1, inplace=True)
trades.set_index('<ID>', inplace=True)
trades.rename(columns= {'<LAST>' : 'price', '<VOL>' : 'qty', '<OPER>' : 'buy_sell'}, inplace=True)
trades.head()



,price,qty,buy_sell,time
<ID>,,,,
10617801085,314.7,10,B,2024-06-06 09:59:37
10617801086,314.7,200,S,2024-06-06 09:59:37
10617801087,314.7,20,S,2024-06-06 09:59:37
10617801088,314.7,10,S,2024-06-06 09:59:37
10617801089,314.7,10,S,2024-06-06 09:59:37


In [64]:
def get_bars(df, add_time=False):
    ohlc = df["price"].ohlc()
    # volume-weighted average price (VWAP)
    #vwap = (
    #    df.apply(lambda x: np.average(x["price"], weights=x["qty"]), include_groups=False)
    #    .to_frame("vwap")
    #)
    vol = df["qty"].sum().to_frame("vol")
    cnt = df["qty"].size().to_frame("cnt")
    if add_time:
        time = df["time"].last().to_frame("time")
        res = pd.concat([time, ohlc, vol, cnt], axis=1)
    else:
        res = pd.concat([ohlc, vol, cnt], axis=1)
    return res




In [66]:
# time bars
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases
df_grouped_time = trades.groupby(pd.Grouper(key="time", freq="1min"))
time_bars = get_bars(df_grouped_time)
time_bars


,open,high,low,close,vol,cnt
time,,,,,,
2024-06-06 09:59:00,314.70,314.70,314.70,314.70,24320,140
2024-06-06 10:00:00,314.70,314.89,314.30,314.35,129390,348
2024-06-06 10:01:00,314.30,314.36,313.98,314.23,72650,250
2024-06-06 10:02:00,314.24,315.17,314.23,315.08,77470,235
2024-06-06 10:03:00,315.12,315.16,314.71,314.94,39900,167
...,...,...,...,...,...,...
2024-06-06 19:55:00,313.10,313.10,313.09,313.10,5450,28
2024-06-06 19:56:00,313.10,313.10,313.09,313.10,2440,20
2024-06-06 19:57:00,313.09,313.10,313.07,313.09,5200,30


In [59]:
# tick bars

bar_size = 100 # количество трейдов в одном баре

trades["tick_group"] = (
    pd.Series(list(range(len(trades))))
    .div(bar_size)
    .apply(np.floor)
    .astype(int)
    .values
)

df_grouped_ticks = trades.groupby("tick_group")
tick_bars = get_bars(df_grouped_ticks, add_time=True)

tick_bars

,time,open,high,low,close,vwap,vol,cnt
tick_group,,,,,,,,
0,2024-06-06 09:59:37,314.70,314.70,314.70,314.70,314.700000,16660,100
1,2024-06-06 10:00:06,314.70,314.87,314.70,314.87,314.745886,21100,100
2,2024-06-06 10:00:19,314.80,314.89,314.67,314.67,314.787801,30100,100
3,2024-06-06 10:00:46,314.66,314.85,314.52,314.52,314.678370,59820,100
4,2024-06-06 10:01:03,314.52,314.52,314.30,314.32,314.401474,28350,100
...,...,...,...,...,...,...,...,...
687,2024-06-06 19:49:59,313.09,313.21,313.06,313.09,313.100667,60090,100
688,2024-06-06 19:51:52,313.09,313.16,313.08,313.09,313.111244,29190,100
689,2024-06-06 19:55:10,313.09,313.14,313.02,313.10,313.070327,32120,100


In [76]:
# volume bars
bar_size = 50000 # vol в каждом баре

trades["cum_qty"] = trades["qty"].cumsum()

trades["vol_group"] = (
    trades["cum_qty"]
    .div(bar_size)
    .apply(np.floor)
    .astype(int)
    .values
)

df_grouped_ticks = trades.groupby("vol_group")
volume_bars = get_bars(df_grouped_ticks, add_time=True)
volume_bars



,time,open,high,low,close,vol,cnt
vol_group,,,,,,,
0,2024-06-06 10:00:13,314.70,314.89,314.70,314.81,49290,261
1,2024-06-06 10:00:44,314.81,314.87,314.61,314.73,50290,95
2,2024-06-06 10:00:54,314.73,314.73,314.34,314.34,50110,103
3,2024-06-06 10:01:34,314.33,314.36,314.00,314.00,47850,197
4,2024-06-06 10:02:39,314.00,314.60,313.98,314.58,52340,165
...,...,...,...,...,...,...,...
531,2024-06-06 19:48:50,312.86,313.04,312.86,313.04,50140,67
532,2024-06-06 19:48:58,313.04,313.24,313.04,313.13,51550,74
533,2024-06-06 19:49:52,313.12,313.12,313.06,313.10,48900,80


In [83]:
# dollar bars

bar_size = 2000000 # Объем в одном баре

trades["cum_value"] = (trades["price"] * trades["qty"]).cumsum()

trades["value_group"] = (
    trades["cum_value"]
    .div(bar_size)
    .apply(np.floor)
    .astype(int)
    .values
)

df_grouped_ticks = trades.groupby("value_group")

dollar_bars = get_bars(df_grouped_ticks, add_time=True)
dollar_bars


,time,open,high,low,close,vol,cnt
value_group,,,,,,,
0,2024-06-06 09:59:37,314.70,314.70,314.70,314.70,6350,40
1,2024-06-06 09:59:37,314.70,314.70,314.70,314.70,5950,28
2,2024-06-06 09:59:37,314.70,314.70,314.70,314.70,6720,56
3,2024-06-06 09:59:37,314.70,314.70,314.70,314.70,5300,16
4,2024-06-06 10:00:03,314.70,314.74,314.70,314.73,6910,20
...,...,...,...,...,...,...,...
4196,2024-06-06 19:55:06,313.09,313.14,313.09,313.09,5750,30
4197,2024-06-06 19:56:33,313.09,313.10,313.09,313.10,6820,38
4198,2024-06-06 19:58:40,313.10,313.10,313.07,313.09,6600,52
